In [ ]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import random_rotation
from tensorflow.image import random_crop
from tensorflow.image import random_contrast
from tensorflow.image import random_brightness
from tensorflow.image import random_flip_left_right
from tensorflow.image import random_hue
from tensorflow.image import random_jpeg_quality
from tensorflow.image import random_saturation
from tensorflow.image import central_crop
from tensorflow.errors import InvalidArgumentError
import matplotlib.pyplot as plt
import random
import tensorflow as tf

In [ ]:
def set_all_seeds(seed):
    os.environ["PL_GLOBAL_SEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    tf.experimental.numpy.random.seed(seed)
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ["PYTHONHASHSEED"] = str(seed)

set_all_seeds(420)

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import pandas as pd

classes = pd.read_csv('gdrive/MyDrive/flower_data/cat_to_name.json', header=None)

In [ ]:
classes

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,100,101
0,"{""21"": ""fire lily""","""3"": ""canterbury bells""","""45"": ""bolero deep blue""","""1"": ""pink primrose""","""34"": ""mexican aster""","""27"": ""prince of wales feathers""","""7"": ""moon orchid""","""16"": ""globe-flower""","""25"": ""grape hyacinth""","""26"": ""corn poppy""",...,"""78"": ""lotus""","""88"": ""cyclamen""","""94"": ""foxglove""","""81"": ""frangipani""","""74"": ""rose""","""89"": ""watercress""","""73"": ""water lily""","""46"": ""wallflower""","""77"": ""passion flower""","""51"": ""petunia""}"


In [ ]:
X = {}

for directory in os.listdir('gdrive/MyDrive/flower_data/train'):
    print(directory)
    X[directory] = []
    for i, file in enumerate(os.listdir('gdrive/MyDrive/flower_data/train/' + directory)):
        with Image.open('gdrive/MyDrive/flower_data/train/' + directory + '/' + file) as img:
            X[directory] += [img.copy()]

In [ ]:
no_images_per_class = [len(x) for x in X.values()]
print(np.sum(no_images_per_class))
print(max(no_images_per_class))
print(min(no_images_per_class))

6552
206
27


In [ ]:
def show(img):
    display(Image.fromarray(img))

In [ ]:
import random
X_val = {}


for key, val in X.items():
  val_samp = random.sample(X[key], 7)
  X_val[key] = val_samp

X_indexes = {}

for key, val in X_val.items():
  flower = X[key]
  indexes = []
  for image in val:
      idx = flower.index(image)
      indexes.append(idx)
  X_indexes[key] = indexes

print(X_indexes)

In [ ]:
for key, value in X.items():
  dest = "gdrive/MyDrive/flower_data/train_reduced/{}/".format(key)
  os.makedirs(os.path.dirname(dest), exist_ok=True)
  i = 0
  for image in value:
    if image not in X_val[key]:
      image.save(dest + str(i),'jpeg')
    i += 1
  print(key)

In [ ]:
for key, value in X_val.items():
  dest = "gdrive/MyDrive/flower_data/val/{}/".format(key)
  os.makedirs(os.path.dirname(dest), exist_ok=True)
  i = 0
  for image in value:
    image.save(dest + str(i),'jpeg')
    i += 1
  print(key)

In [ ]:
def count_elements_in_directories(root_directory='train'):
  number_of_elements_in_classes = []
  for directory in os.listdir('gdrive/MyDrive/flower_data/' + root_directory):
    current_directory = os.listdir('gdrive/MyDrive/flower_data/' + root_directory + '/' + directory)
    files_number = len(current_directory)
    number_of_elements_in_classes.append(files_number)
    print(f'{directory}: {files_number}')

  return number_of_elements_in_classes

In [ ]:
train = count_elements_in_directories()
print()
train_reduced = count_elements_in_directories('train_reduced')
print()
for i in range(len(train)):
  print(train[i] - train_reduced[i])

In [ ]:
def get_crop_size(img, minprop=0.7):
    def rand_prop(x):
        return x + (1. - x) * np.random.random()

    height, width, chanells = img.shape
    height_prop = 1.
    width_prop = 1.

    if height > width:
        height_prop = rand_prop(minprop)
        width_prop = rand_prop(height_prop)
    else:
        width_prop = rand_prop(minprop)
        height_prop = rand_prop(width_prop)

    height = np.floor(height_prop * height).astype(int)
    width = np.floor(width_prop * width).astype(int)
    return (height, width, chanells)

In [ ]:
def apply_random_transforms(img):
    img = random_rotation(img, 20, row_axis=0, col_axis=1, channel_axis=2, fill_mode='reflect')
    img = random_contrast(img, 0.8, 1.2)
    img = random_brightness(img, 0.08)
    img = random_hue(img, 0.025)
    img = random_saturation(img, 0.85, 1.15)
    img = random_jpeg_quality(img, 75, 95)
    img = random_flip_left_right(img)
    return random_crop(img, get_crop_size(img, 0.67)).numpy()

def get_augmented_image(img):
    return Image.fromarray(apply_random_transforms(np.array(img))).resize((224, 224))

In [ ]:
def get_augmented_image(img):
    return Image.fromarray(apply_random_transforms(np.array(img.convert('RGB')))).resize((224, 224))

In [ ]:
all_dirs = []

for directory in os.listdir('gdrive/MyDrive/flower_data/train_reduced'):
    all_dirs += [directory]

In [ ]:
for i, plant in enumerate(all_dirs):
    print("[{:03}] '{}'".format(i, plant))
    original_photos = []
    photos_names = []

    for j, file in enumerate(os.listdir('gdrive/MyDrive/flower_data/train_reduced/' + plant)):
        with Image.open('gdrive/MyDrive/flower_data/train_reduced/' + plant + '/' + file) as img:
            original_photos += [img.copy()]
            photos_names += ['gdrive/MyDrive/flower_data/train_reduced/' + plant + '/' + file]

    dest = "gdrive/MyDrive/flower_data/augmented_new/train/{}/".format(plant)
    os.makedirs(os.path.dirname(dest), exist_ok=True)

    counter = 1

    for j, img in enumerate(original_photos):
        try:
            img.convert('RGB').resize((224, 224)).save(
                dest + "{}_{}.jpg".format(plant.replace(' ', '_'), counter)
            )
            counter += 1
            print("\r{}".format(counter), end='')
        except:
            print("\nError encountered: ", photos_names[j])
            print(e)

    n_items = len(original_photos)
    while counter <= 100:
        try:
            get_augmented_image(original_photos[np.random.randint(n_items)]).save(
                dest + "{}_{}.jpg".format(plant.replace(' ', '_'), counter)
            )
            counter += 1
            print("\r{}".format(counter), end='')
        except Exception as e:
            print("\nError encountered: ", photos_names[j])
            print(e)

In [ ]:
for i, plant in enumerate(all_dirs):
    print("[{:03}] '{}'".format(i, plant))
    original_photos = []
    photos_names = []

    for j, file in enumerate(os.listdir('gdrive/MyDrive/flower_data/val/' + plant)):
        with Image.open('gdrive/MyDrive/flower_data/val/' + plant + '/' + file) as img:
            original_photos += [img.copy()]
            photos_names += ['gdrive/MyDrive/flower_data/val/' + plant + '/' + file]

    dest = "gdrive/MyDrive/flower_data/augmented/val/{}/".format(plant)
    os.makedirs(os.path.dirname(dest), exist_ok=True)

    counter = 1

    for j, img in enumerate(original_photos):
        try:
            img.convert('RGB').resize((224, 224)).save(
                dest + "{}_{}.jpg".format(plant.replace(' ', '_'), counter)
            )
            counter += 1
            print("\r{}".format(counter), end='')
        except:
            print("\nError encountered: ", photos_names[j])
            print(e)

In [ ]:
for i, plant in enumerate(all_dirs):
    print("[{:03}] '{}'".format(i, plant))
    original_photos = []
    photos_names = []

    for j, file in enumerate(os.listdir('gdrive/MyDrive/flower_data/test/' + plant)):
        with Image.open('gdrive/MyDrive/flower_data/test/' + plant + '/' + file) as img:
            original_photos += [img.copy()]
            photos_names += ['gdrive/MyDrive/flower_data/test/' + plant + '/' + file]

    dest = "gdrive/MyDrive/flower_data/augmented/test/{}/".format(plant)
    os.makedirs(os.path.dirname(dest), exist_ok=True)

    counter = 1

    for j, img in enumerate(original_photos):
        try:
            img.convert('RGB').resize((224, 224)).save(
                dest + "{}_{}.jpg".format(plant.replace(' ', '_'), counter)
            )
            counter += 1
            print("\r{}".format(counter), end='')
        except:
            print("\nError encountered: ", photos_names[j])
            print(e)